# Import aller nötigen Module

In [57]:
import os

import numpy as np
import pandas as pd
#Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Filtering
from scipy.signal import butter, filtfilt
from scipy import stats

# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Hilfsfunktionen
Hier definieren wir unsere Hilfsfunktionen zum Plotten

In [58]:
#Vordefinierte Hilfsfunktion zum plotten
def plot_filter(dataframe, data_column_name, filtered_column_name, title):
    fig = px.line(dataframe, x='Time (s)', y=[data_column_name, filtered_column_name], title=title)
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=filtered_column_name)
    fig.show() 

# Datenimport

Hier laden wir unsere .pyphox Dateien und fangen ab, falls diese nicht vorhanden sein solte. Zudem setzen wir schoonmal die Spaltennamen fest um diese im fortlaufenen Programm nutzen zu können

In [59]:

# Error-Handling
try:
    dataframeHammer = pd.read_csv("RawDataHammer.csv")
    dataframeWischen = pd.read_csv("RawDataWisch.csv")
except FileNotFoundError:
    print(f"Error: File not found")
    exit()


# Spaltennamen festsetzen: 
time_col = "Time (s)"
gx_col = "Linear Acceleration x (m/s^2)"
gy_col = "Linear Acceleration y (m/s^2)"
gz_col = "Linear Acceleration z (m/s^2)"
abs_col = "Absolute acceleration (m/s^2)"

# Preview of Data
print("Dataset Hammer:")
print(dataframeHammer.head())
print("Dataset Wischen:")
print(dataframeWischen.head())


Dataset Hammer:
   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.095744                          0.001                          0.060   
1  0.100726                          0.001                          0.031   
2  0.105707                          0.000                         -0.001   
3  0.110688                         -0.003                         -0.053   
4  0.115670                         -0.005                         -0.078   

   Linear Acceleration z (m/s^2)  Absolute acceleration (m/s^2)  
0                         -0.015                       0.061855  
1                         -0.007                       0.031796  
2                          0.000                       0.001000  
3                          0.014                       0.054900  
4                          0.020                       0.080678  
Dataset Wischen:
   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.093435                    

# Frequenzfilter

Hier wird das Rauschen reduziert. Dafür nutzen wir Butterworth aus der Vorlesung. Damit werden hoch- bzw. niederfrequente Signale gefiltert. 
Zuerst folgen die niederfrequenten Signale, dann die hochfrequenten:

In [60]:
filtered_dataframeHammer = dataframeHammer.copy()
filtered_dataframeWischen = dataframeWischen.copy()

filter_order = 4  

low_cutoff = 0.5
high_cutoff = 5 

# Hier nutzen wir den butterwort filter um die Daten zu glätten
b, a = butter(filter_order, [low_cutoff, high_cutoff], btype='band', analog=False, fs =200)

# Apply the filter to acceleration signals
filtered_dataframeWischen['filtered_X_band'] = filtfilt(b, a, dataframeWischen['Linear Acceleration x (m/s^2)'])
filtered_dataframeWischen['filtered_Y_band'] = filtfilt(b, a, dataframeWischen['Linear Acceleration y (m/s^2)'])
filtered_dataframeWischen['filtered_Z_band'] = filtfilt(b, a, dataframeWischen['Linear Acceleration z (m/s^2)'])

plot_filter(filtered_dataframeWischen, 'Linear Acceleration x (m/s^2)', 'filtered_X_band', 'Acceleration X (Wischen) with Band-Pass Filtering')
plot_filter(filtered_dataframeWischen, 'Linear Acceleration y (m/s^2)', 'filtered_Y_band', 'Acceleration Y (Wischen) with Band-Pass Filtering')
plot_filter(filtered_dataframeWischen, 'Linear Acceleration z (m/s^2)', 'filtered_Z_band', 'Acceleration Z (Wischen) with Band-Pass Filtering')

filtered_dataframeHammer['filtered_X_band'] = filtfilt(b, a, dataframeHammer['Linear Acceleration x (m/s^2)'])
filtered_dataframeHammer['filtered_Y_band'] = filtfilt(b, a, dataframeHammer['Linear Acceleration y (m/s^2)'])
filtered_dataframeHammer['filtered_Z_band'] = filtfilt(b, a, dataframeHammer['Linear Acceleration z (m/s^2)'])

plot_filter(filtered_dataframeHammer, 'Linear Acceleration x (m/s^2)', 'filtered_X_band', 'Acceleration X (Hammer) with Band-Pass Filtering')
plot_filter(filtered_dataframeHammer, 'Linear Acceleration y (m/s^2)', 'filtered_Y_band', 'Acceleration Y (Hammer) with Band-Pass Filtering')
plot_filter(filtered_dataframeHammer, 'Linear Acceleration z (m/s^2)', 'filtered_Z_band', 'Acceleration Z (Hammer) with Band-Pass Filtering')

# Ausreißer Bereinigung
Hier wollen wir nun die möglichen Ausreißer bereinigen: 

In [61]:
# Berechnung der Z-Scores für jede Spalte (X, Y, Z)
filtered_dataframeWischen['Z_Score_X'] = stats.zscore(filtered_dataframeWischen['Linear Acceleration x (m/s^2)'])
filtered_dataframeWischen['Z_Score_Y'] = stats.zscore(filtered_dataframeWischen['Linear Acceleration y (m/s^2)'])
filtered_dataframeWischen['Z_Score_Z'] = stats.zscore(filtered_dataframeWischen['Linear Acceleration z (m/s^2)'])

filtered_dataframeHammer['Z_Score_X'] = stats.zscore(filtered_dataframeHammer['Linear Acceleration x (m/s^2)'])
filtered_dataframeHammer['Z_Score_Y'] = stats.zscore(filtered_dataframeHammer['Linear Acceleration y (m/s^2)'])
filtered_dataframeHammer['Z_Score_Z'] = stats.zscore(filtered_dataframeHammer['Linear Acceleration z (m/s^2)'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 4  

# Filtern der Ausreißer aus dem filterd_dataframe (nur wenn alle drei Achsen unterhalb des Threshold liegen sind Daten ok)
filtered_dataframe = filtered_dataframeWischen[(abs(filtered_dataframeWischen['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframeWischen['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframeWischen['Z_Score_Z']) <= z_score_threshold)]



# Jetzt enthält 'filtered_dataframe' nur noch die Datenpunkte, die nicht als Ausreißer betrachtet werden.
print(filtered_dataframeWischen.shape)
print(filtered_dataframeHammer.shape)

#Plotting
plot_filter(filtered_dataframeWischen, 'Linear Acceleration x (m/s^2)', 'Z_Score_X', 'Acceleration (Wischen) X with Z-Score Filtering')
plot_filter(filtered_dataframeWischen, 'Linear Acceleration y (m/s^2)', 'Z_Score_Y', 'Acceleration (Wischen) Y with Z-Score Filtering')
plot_filter(filtered_dataframeWischen, 'Linear Acceleration z (m/s^2)', 'Z_Score_Z', 'Acceleration (Wischen) Z with Z-Score Filtering')

plot_filter(filtered_dataframeHammer, 'Linear Acceleration x (m/s^2)', 'Z_Score_X', 'Acceleration (Hammer) X with Z-Score Filtering')
plot_filter(filtered_dataframeHammer, 'Linear Acceleration y (m/s^2)', 'Z_Score_Y', 'Acceleration (Hammer) Y with Z-Score Filtering')
plot_filter(filtered_dataframeHammer, 'Linear Acceleration z (m/s^2)', 'Z_Score_Z', 'Acceleration (Hammer) Z with Z-Score Filtering')


(3998, 11)
(3997, 11)


# Windowing
Hier nutzen wir Windowing um die Daten zu zerlegen. Das brauchen wir später für die Features-Berechnung

In [62]:
# Da fs = 200 Hz setzen wir das Fenster auf 0.5 Sekunden um auch kurze Bewegungsereignisse abzubilden. 
# Wir könnten window_size aber auch auf 200 setzen.
window_size = 100 
overlap = 50 

#Liste initiieren für Fenster
windows_wischen = []

#Für jedes Fenster in den Daten neuen Dataframe erstellen, der in Liste abgelegt wird 
for i in range(0, len(filtered_dataframeWischen), window_size - overlap):
    window = filtered_dataframeWischen.iloc[i:i+window_size]
    windows_wischen.append(window)

windows_hammer = []

for i in range(0, len(filtered_dataframeHammer), window_size - overlap):
    window = filtered_dataframeHammer.iloc[i:i+window_size]
    windows_hammer.append(window)




#Ersten Dataframe in der Liste ausgeben lassen 
print("Wischen: ")
print(windows_wischen[0].head())
print("Hammer: ")
print(windows_hammer[0].head())
#Anzahl Datenpunkte des ersten Dataframes ausgeben lassen 
print("Wischen: ")
print(len(windows_wischen[0]))
print("Hammer: ")
print(len(windows_hammer[0]))
#Spalten des ersten Dataframes ausgeben lassen 
print("Wischen: ")
print(windows_wischen[0].columns)
print("Hammer: ")
print(windows_hammer[0].columns)

#Ausgeben lassen wie viele Dataframes also Fenster entstanden sind 
print("Wischen: ")
print (len(windows_wischen))
print("Hammer: ")
print (len(windows_hammer))


Wischen: 
   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.093435                          0.003                          0.139   
1  0.098417                          0.003                          0.143   
2  0.103398                          0.003                          0.129   
3  0.108380                          0.003                          0.115   
4  0.113361                          0.003                          0.098   

   Linear Acceleration z (m/s^2)  Absolute acceleration (m/s^2)  \
0                         -0.007                       0.139208   
1                         -0.008                       0.143255   
2                         -0.007                       0.129225   
3                         -0.006                       0.115195   
4                         -0.006                       0.098229   

   filtered_X_band  filtered_Y_band  filtered_Z_band  Z_Score_X  Z_Score_Y  \
0        -2.567775         1.750328        -0.

# Features erstellen
Hier berechnen wir die Merkmale, die wesentliche Eigneschaften der Daten kompakt beschreiben. Diese dienen als Eingabe für den späteren Entscheidungsbaum. 

In [63]:
# FEATURES WISCHEN

#Leere Liste erstellen, um Features zu speichern 
feature_list_wischen = []

#Jeden Dataframe in Windows einzeln durchgehen und für jede der drei Achsen die Features berechnen 
for window in windows_wischen:
    acceleration_data = window[['Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)']]
    #Mittelwert
    acceleration_mean = acceleration_data.mean()
    #Standardabweichung
    acceleration_std_dev = acceleration_data.std()
    #Minimum
    acceleration_min = acceleration_data.min()
    #Maximum
    acceleration_max = acceleration_data.max()
    
    #Berechnete Werte an Liste anhängen, unter entsprechend benannten Spalten 
    feature_list_wischen.append({
        'BeschleunigungX_Mittelwert': acceleration_mean.iloc[0],
        'BeschleunigungY_Mittelwert': acceleration_mean.iloc[1],
        'BeschleunigungZ_Mittelwert': acceleration_mean.iloc[2],
        'BeschleunigungX_Standardabweichung': acceleration_std_dev.iloc[0],
        'BeschleunigungY_Standardabweichung': acceleration_std_dev.iloc[1],
        'BeschleunigungZ_Standardabweichung': acceleration_std_dev.iloc[2],
        'BeschleunigungX_Minimum': acceleration_min.iloc[0],
        'BeschleunigungY_Minimum': acceleration_min.iloc[1],
        'BeschleunigungZ_Minimum': acceleration_min.iloc[2],
        'BeschleunigungX_Maximum': acceleration_max.iloc[0],
        'BeschleunigungY_Maximum': acceleration_max.iloc[1],
        'BeschleunigungZ_Maximum': acceleration_max.iloc[2]
    })

#In einen Pandas Dataframe konvertieren und ausgeben
feature_df_wischen = pd.DataFrame(feature_list_wischen)
# Hier Labeln wir die Feature Zeile für den Entscheidungsbaum
feature_df_wischen["label"] = 0 
print("Wischen: ")
print(feature_df_wischen.columns)

#Dimensionen der Feautures ausgeben lassen 
feature_df_wischen.shape


#FEATURES HAMMER 

#Leere Liste erstellen, um Features zu speichern 
feature_list_hammer = []

#Jeden Dataframe in Windows einzeln durchgehen und für jede der drei Achsen die Features berechnen 
for window in windows_hammer:
    acceleration_data = window[['Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)']]
    #Mittelwert
    acceleration_mean = acceleration_data.mean()
    #Standardabweichung
    acceleration_std_dev = acceleration_data.std()
    #Minimum
    acceleration_min = acceleration_data.min()
    #Maximum
    acceleration_max = acceleration_data.max()
    
    #Berechnete Werte an Liste anhängen, unter entsprechend benannten Spalten 
    feature_list_hammer.append({
        'BeschleunigungX_Mittelwert': acceleration_mean.iloc[0],
        'BeschleunigungY_Mittelwert': acceleration_mean.iloc[1],
        'BeschleunigungZ_Mittelwert': acceleration_mean.iloc[2],
        'BeschleunigungX_Standardabweichung': acceleration_std_dev.iloc[0],
        'BeschleunigungY_Standardabweichung': acceleration_std_dev.iloc[1],
        'BeschleunigungZ_Standardabweichung': acceleration_std_dev.iloc[2],
        'BeschleunigungX_Minimum': acceleration_min.iloc[0],
        'BeschleunigungY_Minimum': acceleration_min.iloc[1],
        'BeschleunigungZ_Minimum': acceleration_min.iloc[2],
        'BeschleunigungX_Maximum': acceleration_max.iloc[0],
        'BeschleunigungY_Maximum': acceleration_max.iloc[1],
        'BeschleunigungZ_Maximum': acceleration_max.iloc[2]
    })

#In einen Pandas Dataframe konvertieren und ausgeben
feature_df_hammer = pd.DataFrame(feature_list_hammer)

# Hier Labeln wir die Feature Zeile für den Entscheidungsbaum
feature_df_hammer["label"] = 1 
print("Hammer: ")
print(feature_df_hammer.columns)

#Dimensionen der Feautures ausgeben lassen 
feature_df_hammer.shape


Wischen: 
Index(['BeschleunigungX_Mittelwert', 'BeschleunigungY_Mittelwert',
       'BeschleunigungZ_Mittelwert', 'BeschleunigungX_Standardabweichung',
       'BeschleunigungY_Standardabweichung',
       'BeschleunigungZ_Standardabweichung', 'BeschleunigungX_Minimum',
       'BeschleunigungY_Minimum', 'BeschleunigungZ_Minimum',
       'BeschleunigungX_Maximum', 'BeschleunigungY_Maximum',
       'BeschleunigungZ_Maximum', 'label'],
      dtype='object')
Hammer: 
Index(['BeschleunigungX_Mittelwert', 'BeschleunigungY_Mittelwert',
       'BeschleunigungZ_Mittelwert', 'BeschleunigungX_Standardabweichung',
       'BeschleunigungY_Standardabweichung',
       'BeschleunigungZ_Standardabweichung', 'BeschleunigungX_Minimum',
       'BeschleunigungY_Minimum', 'BeschleunigungZ_Minimum',
       'BeschleunigungX_Maximum', 'BeschleunigungY_Maximum',
       'BeschleunigungZ_Maximum', 'label'],
      dtype='object')


(80, 13)

# Features zusammenführen

Wir haben momentan 2 verschiedene Feature-Listen. Diese wollen wir zusammenführen. Der Baum kann anhand der Label (0 für Wischen und 1 für Hammer) die Features der jeweiligen Datei erkennen. 

In [64]:
feature_df = pd.concat([feature_df_wischen, feature_df_hammer], ignore_index=True)
print(feature_df.columns)

Index(['BeschleunigungX_Mittelwert', 'BeschleunigungY_Mittelwert',
       'BeschleunigungZ_Mittelwert', 'BeschleunigungX_Standardabweichung',
       'BeschleunigungY_Standardabweichung',
       'BeschleunigungZ_Standardabweichung', 'BeschleunigungX_Minimum',
       'BeschleunigungY_Minimum', 'BeschleunigungZ_Minimum',
       'BeschleunigungX_Maximum', 'BeschleunigungY_Maximum',
       'BeschleunigungZ_Maximum', 'label'],
      dtype='object')


## Entscheidungsbaum

Hier trainieren wir den Entscheidungsbaum

In [65]:
# Label für Datensatz setzen 
feature_df["label"] = "Wischen" # Das ändern wenn anderer Datensatz

# Features und Zielvariable trennen
X = feature_df.drop(columns=["label"])
Y = feature_df["label"]

# Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

#Entscheidungsbaum(CART)

clf = DecisionTreeClassifier(
    criterion="gini", 
    max_depth=5,
    min_samples_leaf=5,
    random_state=42
)

#Training
clf.fit(X_train, y_train)

#Validierung
y_pred = clf.predict(X_test)

print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion Matrix", confusion_matrix(y_test, y_pred))
print("Classification Report" + classification_report(y_test, y_pred))


# Entscheidungsbaum speichern

import joblib

joblib.dump(clf, "decision_tree_model.pkl")
print("Modell gespeichert")




Accuracy 1.0
Confusion Matrix [[32]]
Classification Report              precision    recall  f1-score   support

     Wischen       1.00      1.00      1.00        32

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

Modell gespeichert


## Entscheidungsbaum laden und neue Daten klassifizieren

Hier laden wir den trainierten Entscheidungsbaum und nutzen diesen zu Klassifikation von neuen Sensordaten

In [66]:
# Modell laden
model = joblib.load("decision_tree_model.pkl")

# Beispiel: erstes Feature-Fenster klassifizieren 
sample = X.iloc[[0]]
prediction = model.predict(sample)

print("Vorhergesagte Klasse:", prediction[0])




Vorhergesagte Klasse: Wischen


## Dobot Aktionen anhand der Vorhersage ausführen

Hier verknüpfen wir die prediction mit dem Bot

In [67]:
import sys
import os
sys.path.insert(0, os.path.abspath('.'))

from lib.dobot import Dobot

bot = Dobot('COM3')

def wischen(bot):
    print("Aktion: Wischen")
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_haken, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Kastenstartposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kastenposition 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kastenposition 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kastenposition 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kastenposition 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kastenposition 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    print('Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    print('Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    print('Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Zurück nach Position 1')
    bot.move_to(x_max, y_min, z_max, 0)

def hammer(bot):

    print("Aktion: Hammer")
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_max, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    print('Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Hochfahren 3')
    bot.move_to(x_min, y_min, z_max, 0)

if prediction[0] == "Hammer":
    wischen(bot)
else:
    hammer(bot)

SerialException: could not open port 'COM3': FileNotFoundError(2, 'Das System kann die angegebene Datei nicht finden.', None, 2)